In [3]:
import polars as pl
from xlsx_generator import fetch_csv

ticket = "30088f13cc4a64c91aef019587bf2a31f7ff7055306e11abaef001d927dd099a"
auth = ("st101885", "x0301093100")

fakulta="PRF"

params_kateder = {
    "typPracoviste":"K",
    "zkratka":"%",
    "nadrazenePracoviste":fakulta
    }
print(params_kateder["nadrazenePracoviste"])
katedry_csv = pl.read_csv(fetch_csv(service="/ciselniky/getSeznamPracovist", params_plus=params_kateder, ticket=ticket, auth=auth), separator=";")
#print(katedry_csv.head())
katedry_list = katedry_csv.to_series(2)
katedry_list = katedry_list.to_list()
#print(katedry_list)


loner = katedry_list.pop(0)
params_rozvrh = {
    "stagUser": "F23112",
    "semestr":"%",
    "vsechnyCasyKonani":"false",
    "jenRozvrhoveAkce":"true",
    "vsechnyAkce":"false",
    "jenBudouciAkce":"false",
    "lang":"cs",
    "katedra":loner,
    "rok":"2023"
}
params_predmety = {
    "lang":"cs",
    "katedra":loner,
    "jenNabizeneECTSPrijezdy":"false",
    "rok":"2023"
}

excel_rozvrhy = pl.read_csv(fetch_csv(service="/rozvrhy/getRozvrhByKatedra", params_plus=params_rozvrh, ticket=ticket, auth=auth), separator=";")
excel_predmety = pl.read_csv(fetch_csv(service="/predmety/getPredmetyByKatedraFullInfo", params_plus=params_predmety), separator=";")
#excel_predmety.write_csv("source_testing/predmety-1.csv")
excel_predmety.head(5)

PRF


katedra,zkratka,rok,nazev,nazevDlouhy,maVyuku,vyukaZS,vyukaLS,jakCastoJeNabizen,jakCastoJeNabizenUpresneni,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,…,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,semestr,pocetStudentu,aMax,bMax,cMax,aSkut,bSkut,cSkut
str,str,i64,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,i64,str,str,str,i64,i64,i64
"""CNB""","""BK502""",2023,"""Fyziologie živ…","""Fyziologie živ…","""A""","""A""","""N""","""K""","""""",6,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…","""8024""","""'Mgr. Marian R…","""'Mgr. Marian R…","""5275, 8024""","""'Mgr. Marian R…","""'Mgr. Marian R…","""5275, 8024""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",…,"""HOD/SEM""",0,"""HOD/SEM""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Písemný zápočt…","""1. Vnitřní pro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""","""""","""ZS""",0,"""""","""""","""""",0,0,0
"""CNB""","""BP2""",2023,"""Bakalářská prá…","""Bakalářská prá…","""A""","""A""","""N""","""K""","""""",5,"""NE""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""KFY/BP2""","""""","""""","""N""",…,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem kurzu je…","""Zápočet""","""NE""","""Kombinovaná""","""Zápočet uděluj…","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""","""""","""ZS""",4,"""""","""""","""""",4,0,0
"""CNB""","""BP502""",2023,"""Fyziologie živ…","""Fyziologie živ…","""A""","""A""","""N""","""K""","""""",6,"""NE""","""""","""'Ing. Stanisla…","""'Ing. Stanisla…","""8024""","""'Mgr. Marian R…","""'Mgr. Marian R…","""5275, 8024""","""'Mgr. Marian R…","""'Mgr. Marian R…","""5275, 8024""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",…,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""ANO""","""Kombinovaná""","""Písemný zápočt…","""1. Vnitřní pro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""","""""","""ZS""",0,"""""","""""","""""",0,0,0
"""CNB""","""EXK""",2023,"""Exkurze""","""Exkurze""","""A""","""A""","""A""","""K""","""""",1,"""NE""","""""","""'doc. Ing. Mar…","""'doc. Ing. Mar…","""2384""","""""","""""","""""","""""","""""","""""","""'doc. Ing. Mar…","""'doc. Ing. Mar…","""2384""","""""","""""","""""","""""","""""","""""","""""","""""","""KFY/EXK""","""""","""""","""N""",…,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem kurzu je…","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""","""""","""ZS""",0,

In [8]:
#for num,katedra in enumerate(katedry_list):
katedra = "PRFD"
params_rozvrh["katedra"] = katedra
params_predmety["katedra"] = katedra

temp_rozvrhy = pl.read_csv(fetch_csv(service="/rozvrhy/getRozvrhByKatedra", params_plus=params_rozvrh, ticket=ticket, manual_login=auth), separator=";")
temp_predmety = pl.read_csv(fetch_csv(service="/predmety/getPredmetyByKatedraFullInfo", params_plus=params_predmety), separator=";")
temp_rozvrhy.head(3)

#     excel_predmety = excel_predmety.extend(temp_predmety)
#     excel_rozvrhy = excel_rozvrhy.extend(temp_rozvrhy)

# excel_rozvrhy.write_csv("source_tables/rozvrhy_fakulta.csv")
# excel_predmety.write_csv("source_tables/predmety_fakulta.csv")

roakIdno,nazev,katedra,predmet,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,semestr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str


In [5]:
type(excel_rozvrhy.dtypes[0])

Int64

In [6]:
def type_check(dataframe1:"pl.DataFrame", dataframe2:"pl.DataFrame") -> list:
    types1 = dataframe1.dtypes
    types2 = dataframe2.dtypes
    problems = []
    for index in range(len(types1) - 1):
        if types1[index] != types2[index]:
            problems.append(dataframe1.columns[index])

    return problems


type_check(excel_rozvrhy, temp_rozvrhy)


['roakIdno',
 'rok',
 'planObsazeni',
 'obsazeni',
 'pocetVyucHodin',
 'tydenOd',
 'tydenDo',
 'referencedIdno']

In [7]:
excel_predmety.vstack(temp_predmety)

SchemaError: cannot extend/append Int64 with String

In [5]:
big_one = pl.read_csv("source_tables/predmety_F23112.csv")
bigger_one = pl.read_csv("source_tables/rozvrhy_fakulta.csv")

big_one.head()

FileNotFoundError: Systém nemůže nalézt uvedený soubor. (os error 2): source_tables/predmety_F23112.csv

In [8]:
excel_rozvrhy.with_columns(
    str(pl.col("rok") + pl.col("viceZapis"))
    ).alias("Hello").select("Hello")

ColumnNotFoundError: [(col("rok")) + (col("viceZapis"))]

Error originated just after this operation:
DF ["roakIdno", "nazev", "katedra", "predmet"]; PROJECT */55 COLUMNS; SELECTION: "None"

In [5]:
def mygen():
    x = 0
    while True:
        x += 1
        yield x

In [ ]:
import xlsx_generator as tablegen
tablegen.pull_data(
    search_type="Katedra",
    search_target="KMA",
    ticket_over="56ac36a08e6d8d1fd3aa7579b23064c3402f963e3b7d3fd1fb2a03197a555050"
)

In [24]:
fakulta = ["PRF", "FŽP"]
typ = ["Bakalářský"]
forma = ["Prezenční"]

params_sp = {"pouzePlatne":True, "lang":"cs"}
params_sp["rok"] = 2024
sp_table = pl.read_csv(fetch_csv("/programy/getStudijniProgramy", params_plus=params_sp), separator=";").select("nazev", "stprIdno", "typ", "forma", "fakulta", "nazevAn")

fak_check = pl.col("fakulta").is_in(fakulta) if len(fakulta) > 0 else True
form_check = pl.col("forma").is_in(forma) if len(forma) > 0 else True
typ_check = pl.col("typ").is_in(typ) if len(typ) > 0 else True

sp_table = sp_table.filter(fak_check & form_check & typ_check).with_columns(pl.concat_str(
    pl.when(pl.col("nazevAn").str.len_chars() > 0).then(pl.col("nazevAn")).otherwise(pl.col("nazev")),
    pl.concat_str(
        pl.col("typ").str.slice(0,1),
        pl.col("forma").str.slice(0,1),
        pl.col("fakulta"),
    separator=","), separator=" - ").alias("Hello")).select("stprIdno", "Hello", "nazevAn")

sp_table

# ids = sp_table.to_series(0).to_list()
# names = sp_table.to_series(1).to_list()
# my_dict = {ids[x]:names[x] for x in range(len(ids))}
# my_dict

stprIdno,Hello,nazevAn
i64,str,str
398,"""Aplikovaná fyz…","""Applied Physic…"
891,"""Aplikovaná geo…","""Applied Geoinf…"
217,"""Aplikovaná inf…","""Applied Inform…"
1509,"""Aplikovaná inf…",""""""
1253,"""Aplikované nan…","""Applied nanote…"
…,…,…
244,"""Matematika - B…","""Mathematics"""
1217,"""Matematika pro…",""""""
803,"""Matematika ve …","""Mathematics in…"
